In [5]:
%load_ext autoreload
%autoreload 2

import os
from entities.simulation import Simulation
from model.TrafficMDP import TrafficMDP

traffic_mdp = TrafficMDP()

simulation = Simulation(
     name="Ambient Test",
     ambient_images_path=[f"./assets/img/{_}" for _ in os.listdir("./assets/img")],
     #audio_effect_path= "./assets/audio/Street Sound Effect.mp3",
     car_spawn_frequency=1.5,  # seconds
     max_simulation_time=120,  # seconds
)

simulation.run(traffic_mdp)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: maintain
State EW, Action: change
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maintain
State NS, Action: maint